In [1]:
import sys
sys.path.append(".")
sys.path.append("../")

In [2]:
from CCG_new import parser

In [3]:
trainer = parser.CCGParserTrainer("re", explanation_file="../data/tacred_explanations.json",
                                  unlabeled_data_file="../data/tacred_dev.json")

In [4]:
trainer.train()

Parser: Loaded explanation data
Parser: Created and Set Grammar
Parser: Tokenized Explanations
Parser: Built Labeling Rules
Parser: Set labeling functions
Parser: Built soft labeling functions
Parser: Done


In [5]:
parser = trainer.get_parser()

In [6]:
loaded_data = parser.loaded_data

In [7]:
len(parser.filtered_raw_explanations)

157

In [8]:
# parser.soft_labeling_functions

In [9]:
# parser.filtered_raw_explanations

In [10]:
# parser.soft_label_function_to_semantic_map

In [8]:
import json
import pickle
import dill

In [21]:
with open("../data/tacred_explanations.json") as f:
    explanations = json.load(f)
    explanations = [ent["explanation"] for ent in explanations]

In [22]:
explanations[120]

"The word 'employees' appears right after OBJ"

In [23]:
labeling_function_indicies = []
for pair in parser.soft_labeling_functions:
    func, _ = pair
    sem = parser.soft_label_function_to_semantic_map[func]
    expl = parser.filtered_raw_explanations[sem]
    index = explanations.index(expl)
    labeling_function_indicies.append(index)
    

In [24]:
len(labeling_function_indicies)

157

In [25]:
labeling_functions = parser.labeling_functions
function_ner_types = parser.ner_types
task = "re"

In [26]:
TACRED_LABEL_MAP = {
    'per:title': 0,
    'org:top_members/employees': 1,
    'per:employee_of': 2,
    'org:alternate_names': 3,
    'org:country_of_headquarters': 4,
    'per:countries_of_residence': 5,
    'org:city_of_headquarters': 6,
    'per:cities_of_residence': 7,
    'per:age': 8,
    'per:stateorprovinces_of_residence': 9,
    'per:origin': 10,
    'org:subsidiaries': 11,
    'org:parents': 12,
    'per:spouse': 13,
    'org:stateorprovince_of_headquarters': 14,
    'per:children': 15,
    'per:other_family': 16,
    'per:alternate_names': 17,
    'org:members': 18,
    'per:siblings': 19,
    'per:schools_attended': 20,
    'per:parents': 21,
    'per:date_of_death': 22,
    'org:member_of': 23,
    'org:founded_by': 24,
    'org:website': 25,
    'per:cause_of_death': 26,
    'org:political/religious_affiliation': 27,
    'org:founded': 28,
    'per:city_of_death': 29,
    'org:shareholders': 30,
    'org:number_of_employees/members': 31,
    'per:date_of_birth': 32,
    'per:city_of_birth': 33,
    'per:charges': 34,
    'per:stateorprovince_of_death': 35,
    'per:religion': 36,
    'per:stateorprovince_of_birth': 37,
    'per:country_of_birth': 38,
    'org:dissolved': 39,
    'per:country_of_death': 40,
    'no_relation': 41
}

In [68]:
# with open("../data/training_data/train_phrases_debug.p", "rb") as f:
#         phrases = pickle.load(f)

        
# matched_data_indicies = []
# soft_matched_indicies = []
# # matched_data_tuples = []
# # unlabeled_data_phrases = []
# labels = []

# for i, phrase in enumerate(phrases):
#     not_matched = True
#     for function in labeling_functions:
#         try:
#             if function(phrase):
#                 if task == "re":
#                     subj_type = phrase.ners[phrase.subj_posi].lower()
#                     obj_type = phrase.ners[phrase.obj_posi].lower()
#                     if function_ner_types[function][0] == subj_type and function_ner_types[function][1] == obj_type:
# #                         sentence = phrase.sentence.replace("SUBJ", "SUBJ-{}".format(phrase.ners[phrase.subj_posi]))
# #                         sentence = sentence.replace("OBJ", "OBJ-{}".format(phrase.ners[phrase.obj_posi]))
# #                         matched_data_tuples.append((sentence, labeling_functions[function]))
#                         labels.append(TACRED_LABEL_MAP[labeling_functions[function]]+1)
#                         matched_data_indicies.append(i)
#                         not_matched = False
#                         break
#                 else:
# #                     matched_data_tuples.append((phrase.sentence, labeling_functions[function]))
#                     matched_data_indicies.append(i)
#                     not_matched = False
#                     break
#         except:
#             continue
#     if not_matched:
#         labels.append(0)
#         soft_matched_indicies.append(i)

In [18]:
# with open("matching_indxes.json", "w") as f:
#     json.dump({
#         "matched_data_indicies" : matched_data_indicies, 
#         "soft_matched_indicies" : soft_matched_indicies
#     }, f)

In [170]:
with open("matching_indxes.json") as f:
    blah = json.load(f)
    matched_data_indicies = blah["matched_data_indicies"]
    soft_matched_indicies = blah["soft_matched_indicies"]

In [171]:
len(matched_data_indicies), len(soft_matched_indicies)

(2745, 65379)

In [172]:
soft_matched_indicies[:10], matched_data_indicies[:10]

([0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [29, 58, 90, 98, 104, 109, 128, 181, 190, 216])

In [11]:
import torch
from models import Find_Module
from training.util_functions import build_custom_vocab
from training.util_functions import set_re_dataset_ner_label_space
from tqdm import tqdm
from training.util_classes import BaseVariableLengthDataset
from training.train_util_functions import build_phrase_input, build_mask_mat_for_batch, batch_type_restrict_re
from training.constants import TACRED_ENTITY_TYPES, TACRED_NERS
from CCG_new.soft_grammar_functions import NER_LABEL_SPACE

In [12]:
save_string = "tacred_glove.840B.300d_-1_-1.0"

with open("../data/training_data/unlabeled_data_{}.p".format(save_string), "rb") as f:
    unlabeled_data = pickle.load(f)

with open("../data/vocabs/vocab_tacred_glove.840B.300d_-1_0.6.p", "rb") as f:
    vocab = pickle.load(f)

with open("../data/training_data/labeling_functions_{}.p".format(save_string), "rb") as f:
    soft_labeling_functions_dict = dill.load(f)

with open("../data/training_data/query_tokens_{}.p".format(save_string), "rb") as f:
    tokenized_queries = pickle.load(f)
    
with open("../data/training_data/word2idx_{}.p".format(save_string), "rb") as f:
    quoted_words_to_index = pickle.load(f)

pad_idx = vocab["<pad>"]

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

custom_vocab = build_custom_vocab("tacred", len(vocab))
custom_vocab_length = len(custom_vocab)

find_module = Find_Module.Find_Module(vocab.vectors, pad_idx, 300, 300,
                                      torch.cuda.is_available(),
                                      custom_token_count=custom_vocab_length)

find_module.load_state_dict(torch.load("../data/saved_models/Find-Module-pt_blah2.p"))

<All keys matched successfully>

In [13]:
len(quoted_words_to_index)

159

In [14]:
set_re_dataset_ner_label_space("tacred", TACRED_NERS)

In [15]:
NER_LABEL_SPACE

{'': 0,
 'NUMBER': 1,
 'DATE': 2,
 'EVENT': 3,
 'FAC': 4,
 'STATE_OR_PROVINCE': 5,
 'LANGUAGE': 6,
 'LAW': 7,
 'LOCATION': 8,
 'MONEY': 9,
 'NORP': 10,
 'ORDINAL': 11,
 'ORGANIZATION': 12,
 'PERCENT': 13,
 'PERSON': 14,
 'PRODUCT': 15,
 'QUANTITY': 16,
 'TIME': 17,
 'WORK_OF_ART': 18,
 '<PAD>': 19,
 'TITLE': 20,
 'COUNTRY': 21,
 'CITY': 22,
 'MISC': 23,
 'DURATION': 24,
 'NATIONALITY': 25,
 'CAUSE_OF_DEATH': 26,
 'CRIMINAL_CHARGE': 27,
 'RELIGION': 28,
 'URL': 29,
 'IDEOLOGY': 30}

In [16]:
find_module = find_module.to(device)

# Get queries ready for Find Module
lfind_query_tokens, _ = BaseVariableLengthDataset.variable_length_batch_as_tensors(tokenized_queries, pad_idx)
lfind_query_tokens = lfind_query_tokens.to(device).detach()

soft_labeling_functions = soft_labeling_functions_dict["function_pairs"]
soft_labeling_function_labels = soft_labeling_functions_dict["labels"]

relation_ner_types = TACRED_ENTITY_TYPES
lower_bound = -20.0

In [17]:
lfind_query_tokens.shape

torch.Size([159, 4])

In [180]:
lfind_matrix = []
for i, batch in enumerate(tqdm(unlabeled_data.as_batches(batch_size=100, shuffle=False))):
        unlabeled_tokens, unlabeled_token_lengths, phrases = batch
        unlabeled_tokens = unlabeled_tokens.to(device)
        with torch.no_grad():
            lfind_output = find_module.soft_matching_forward(unlabeled_tokens.detach(), lfind_query_tokens, lower_bound).cpu() # B x seq_len x Q
            lfind_matrix.append(lfind_output)

654it [12:07,  1.11s/it]


In [181]:
new_lfind_matrix = []
for i, tensor in enumerate(lfind_matrix):
    batch_size, seq_length, _ = tensor.shape
    seq_length = min(seq_length, 110)
    new_tensor = torch.zeros(batch_size, 110, 159)
    new_tensor[:,:seq_length,:] = tensor[:, :seq_length, :]
    new_lfind_matrix.append(new_tensor)

In [182]:
full_matrix = torch.cat(new_lfind_matrix)

In [183]:
full_matrix.shape

torch.Size([65379, 110, 159])

In [184]:
with open("/home/rahul/label_mat.p", "wb") as f:
    pickle.dump(full_matrix, f, protocol=4)

In [82]:
lfind_matrix[0].shape

torch.Size([100, 95, 166])

In [83]:
lfind_matrix[1].shape

torch.Size([100, 75, 166])

In [84]:
lfind_matrix[0][0][0]

tensor([[2.0612e-09, 2.0612e-09, 1.3942e-08,  ..., 4.1497e-08, 2.0612e-09,
         2.0612e-09],
        [2.0612e-09, 2.0612e-09, 2.0612e-09,  ..., 2.0612e-09, 2.0612e-09,
         1.3453e-08],
        [2.0612e-09, 2.0612e-09, 2.0612e-09,  ..., 2.0612e-09, 2.0612e-09,
         2.0612e-09],
        ...,
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00]])

In [27]:
function_scores = {}
for i, batch in enumerate(tqdm(unlabeled_data.as_batches(batch_size=100, shuffle=False))):
        unlabeled_tokens, unlabeled_token_lengths, phrases = batch
        unlabeled_tokens = unlabeled_tokens.to(device)
        phrase_input = build_phrase_input(phrases, pad_idx, "re").to(device).detach()
        b_size, seq_length = unlabeled_tokens.shape
        mask_mat = build_mask_mat_for_batch(seq_length).to(device).detach()
        with torch.no_grad():
            lfind_output = find_module.soft_matching_forward(unlabeled_tokens.detach(), lfind_query_tokens, lower_bound).detach() # B x seq_len x Q

            function_batch_scores = []
            for j, pair in enumerate(soft_labeling_functions):
                func, rel = pair
                func_index = labeling_function_indicies[j]
                batch_scores = func(lfind_output, quoted_words_to_index, mask_mat)(phrase_input).detach() # 1 x B

                type_restrict_multiplier = batch_type_restrict_re(rel, phrase_input, relation_ner_types).detach() # 1 x B
                final_scores = batch_scores * type_restrict_multiplier # 1 x B
                final_scores = final_scores.cpu()
                if func_index in function_scores:
                    function_scores[func_index] = torch.cat([function_scores[func_index], final_scores], dim=1)
                else:
                    function_scores[func_index] = final_scores

0it [00:00, ?it/s]../CCG_new/constants.py:443: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ".root": lambda xs: lambda label_mat,keyword_dict,mask_mat:lambda c: torch.max(torch.tensor(sum([x(label_mat,keyword_dict,mask_mat)(c) for x in xs])-len(xs)+1).to(device),torch.tensor(0.0).to(device)) if type(xs) == tuple else xs(label_mat,keyword_dict,mask_mat)(c),
../CCG_new/soft_grammar_functions.py:386: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.max(torch.tensor(F(nouny, option={'attr':'word','range':-1,'numAppear':1,'cmp':'nlt','onlyCount':False})(label_mat,keyword_dict,mask_mat)(c)+F(nouny, option={'attr':'tokens','range':-1,'numAppear':len(nouny.split()),'cmp':'eq','onlyCou

In [28]:
sum(function_scores[0][0])

tensor(61.2273)

In [ ]:
len(function_scores.keys())

In [34]:
with open("/home/rahul/ziqi/NExT/data/tacred/explanations.json") as f:
    explanations = json.load(f)

In [35]:
explanations

[['per:children',
  "'@And'('@Is'('There','@AtMost'('@between'('@And'('ArgY','ArgX')),'@Num'('3','tokens'))),'@Is'('@Word'(''sSPACEdaughter'),'@between'('@And'('ArgY','ArgX'))))",
  ["SUBJ-PERSON 's daughter OBJ-PERSON"],
  ["'s daughter"]],
 ['per:date_of_birth',
  "'@And'('@Is'('There','@AtMost'('@between'('@And'('ArgY','ArgX')),'@Num'('3','tokens'))),'@Is'('@Word'('wasSPACEborn'),'@between'('@And'('ArgY','ArgX'))))",
  ['SUBJ-PERSON was born OBJ-DATE'],
  ['was born']],
 ['org:parents',
  "'@And'('@Is'('asSPACEpartSPACEofSPACEits','@between'('@And'('ArgY','ArgX'))),'@Is'('There','@AtMost'('@between'('@And'('ArgY','ArgX')),'@Num'('5','tokens'))))",
  ['SUBJ-ORGANIZATION as part of its OBJ-ORGANIZATION'],
  ['as part of its']],
 ['per:cause_of_death',
  "'@And'('@Is'('There','@AtMost'('@between'('@And'('ArgY','ArgX')),'@Num'('5','tokens'))),'@Is'('@Word'('whoSPACEdiedSPACEof'),'@between'('@And'('ArgY','ArgX'))))",
  ['SUBJ-PERSON who died of OBJ-CAUSE_OF_DEATH'],
  ['who died of']],
 

In [78]:
filtered_explanations = []
for index in labeling_function_indicies:
    filtered_explanations.append(explanations[index])

In [79]:
with open("/home/rahul/filtered_explanations.json", "w") as f:
    json.dump(filtered_explanations, f)

In [29]:
soft_scores = torch.cat([function_scores[key] for key in function_scores])

In [30]:
soft_scores.shape

torch.Size([157, 65379])

In [31]:
with open("/home/rahul/soft_match_scores.p", "wb") as f:
    pickle.dump(soft_scores, f)

In [74]:
with open("/home/rahul/tacred_explanation_labels.p", "wb") as f:
    pickle.dump(labels, f)

In [75]:
with open("/home/rahul/matched_data_indicies.p", "wb") as f:
    pickle.dump(matched_data_indicies, f)

In [76]:
with open("/home/rahul/unmatched_data_indicies.p", "wb") as f:
    pickle.dump(soft_matched_indicies, f)

In [102]:
with open("../../../ziqi/NExT/tacred_labeled.json") as f:
    ziqi_index = json.load(f)

In [33]:
with open("/home/rahul/word2idx.p".format(save_string), "wb") as f:
    pickle.dump(quoted_words_to_index, f)

In [103]:
ziqi_index[0]

[29, 'per:title']

In [104]:
ziqi_index = [pair[0] for pair in ziqi_index]
missing_indicies = []
only_mine = []
for index in ziqi_index:
    if index not in matched_data_indicies:
        missing_indicies.append(index)

for index in matched_data_indicies:
    if index not in ziqi_index:
        only_mine.append(index)

In [105]:
missing_indicies

[19939, 22241, 24242, 25205, 26266, 33536, 36371, 42365, 52615, 54521, 60836]

In [106]:
only_mine

[878,
 1407,
 2250,
 2590,
 6577,
 6898,
 7874,
 9153,
 10175,
 10519,
 16700,
 18918,
 19398,
 21057,
 21477,
 22478,
 24647,
 24705,
 34199,
 37446,
 38725,
 40475,
 49302,
 50134,
 53552,
 54322,
 56511,
 61572,
 64373,
 64595,
 64932]

In [116]:
filtered_raw_explanations = parser.filtered_raw_explanations

In [117]:
len(filtered_raw_explanations)

157

In [118]:
# filtered_raw_explanations = list(filtered_raw_explanations.values())

In [129]:
from training.util_functions import extract_queries_from_explanations, tokenize

In [132]:
quoted_words = {}
for i, key in enumerate(filtered_raw_explanations):
    explanation = filtered_raw_explanations[key]
    queries = extract_queries_from_explanations(explanation)
    for query in queries:
        query = " ".join(tokenize(query)).strip()
        if len(query.strip()) > 0:
            quoted_words[query] = 1
    print((explanation, len(quoted_words)))

quoted_words = list(quoted_words.keys())

('The phrase "\'s daughter" links SUBJ and OBJ and there are no more than three words between SUBJ and OBJ', 1)
('SUBJ and OBJ sandwich the phrase "was born" and there are no more than three words between SUBJ and OBJ', 2)
('There are no more than five words between SUBJ and OBJ and "as part of its" appears between SUBJ and OBJ', 3)
('Between SUBJ and OBJ the phrase "who died of" occurs and there are no more than five words between SUBJ and OBJ', 4)
('Between SUBJ and OBJ the phrase "is from" appears and there are no more than four words between SUBJ and OBJ', 5)
('The phrase "\'s grandmother" occurs between SUBJ and OBJ and there are no more than four words between SUBJ and OBJ', 6)
('The phrase "is based in" occurs between SUBJ and OBJ and there are no more than five words between SUBJ and OBJ', 7)
('The term "then known as" occurs between SUBJ and OBJ and there are no more than six words between SUBJ and OBJ', 8)
('SUBJ and OBJ sandwich the phrase "\'s mother" and there are no more 

In [135]:
with open("/home/rahul/quoted_words.json", "w") as f:
    json.dump(quoted_words, f)

In [124]:
list(filtered_raw_explanations.values())

['The phrase "\'s daughter" links SUBJ and OBJ and there are no more than three words between SUBJ and OBJ',
 'SUBJ and OBJ sandwich the phrase "was born" and there are no more than three words between SUBJ and OBJ',
 'There are no more than five words between SUBJ and OBJ and "as part of its" appears between SUBJ and OBJ',
 'Between SUBJ and OBJ the phrase "who died of" occurs and there are no more than five words between SUBJ and OBJ',
 'Between SUBJ and OBJ the phrase "is from" appears and there are no more than four words between SUBJ and OBJ',
 'The phrase "\'s grandmother" occurs between SUBJ and OBJ and there are no more than four words between SUBJ and OBJ',
 'The phrase "is based in" occurs between SUBJ and OBJ and there are no more than five words between SUBJ and OBJ',
 'The term "then known as" occurs between SUBJ and OBJ and there are no more than six words between SUBJ and OBJ',
 'SUBJ and OBJ sandwich the phrase "\'s mother" and there are no more than three words between